In [2]:
!pip install pyspark 

     |████████████████████████████████| 212.3MB 70kB/s 
     |████████████████████████████████| 204kB 19.2MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=e28d5be25c5df64403c9a2f8836965c908701dfaec6a676c389165f95531b1aa
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [37]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd 
from pyspark.sql.functions import year, month

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
sc = SparkContext.getOrCreate()

In [5]:
type(sc)

pyspark.context.SparkContext

In [ ]:
help(sc)

Help on SparkContext in module pyspark.context object:

class SparkContext(builtins.object)
 |  SparkContext(master=None, appName=None, sparkHome=None, pyFiles=None, environment=None, batchSize=0, serializer=PickleSerializer(), conf=None, gateway=None, jsc=None, profiler_cls=<class 'pyspark.profiler.BasicProfiler'>)
 |  
 |  Main entry point for Spark functionality. A SparkContext represents the
 |  connection to a Spark cluster, and can be used to create :class:`RDD` and
 |  broadcast variables on that cluster.
 |  
 |  .. note:: Only one :class:`SparkContext` should be active per JVM. You must `stop()`
 |      the active :class:`SparkContext` before creating a new one.
 |  
 |  .. note:: :class:`SparkContext` instance is not supported to share across multiple
 |      processes out of the box, and PySpark does not guarantee multi-processing execution.
 |      Use threads instead for concurrent processing purpose.
 |  
 |  Methods defined here:
 |  
 |  __enter__(self)
 |      Enable '

In [6]:
dir(sc)

['PACKAGE_EXTENSIONS',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_accumulatorServer',
 '_active_spark_context',
 '_assert_on_driver',
 '_batchSize',
 '_callsite',
 '_checkpointFile',
 '_conf',
 '_dictToJavaMap',
 '_do_init',
 '_encryption_enabled',
 '_ensure_initialized',
 '_gateway',
 '_getJavaStorageLevel',
 '_initialize_context',
 '_javaAccumulator',
 '_jsc',
 '_jvm',
 '_lock',
 '_next_accum_id',
 '_pickled_broadcast_vars',
 '_python_includes',
 '_repr_html_',
 '_serialize_to_jvm',
 '_temp_dir',
 '_unbatched_serializer',
 'accumulator',
 'addFile',
 'addPyFile',
 'appName',
 'applicationId',
 'binaryFiles',
 'bina

In [22]:
sqlContext = SQLContext(sc)
li = [('Alice', 1)]
df = sqlContext.createDataFrame(li, ['name', 'age'])
df.collect()

[Row(name='Alice', age=1)]

In [63]:
path_to_data = ''

In [14]:
tesla_rdd = sc.textFile(path_to_data + '/TSLA.csv')

In [15]:
tesla_rdd.take(5)

['Date,Open,High,Low,Close,AdjClose,Volume',
 '2019-07-15,248.000000,254.419998,244.860001,253.500000,253.500000,11000100',
 '2019-07-16,249.300003,253.529999,247.929993,252.380005,252.380005,8149000',
 '2019-07-17,255.669998,258.309998,253.350006,254.860001,254.860001,9764700',
 '2019-07-18,255.050003,255.750000,251.889999,253.539993,253.539993,4764500']

In [16]:
type(tesla_rdd)

pyspark.rdd.RDD

In [17]:
csv_rdd = tesla_rdd.map(lambda row: row.split(","))

In [20]:
csv_rdd.take(5)

[['Date', 'Open', 'High', 'Low', 'Close', 'AdjClose', 'Volume'],
 ['2019-07-15',
  '248.000000',
  '254.419998',
  '244.860001',
  '253.500000',
  '253.500000',
  '11000100'],
 ['2019-07-16',
  '249.300003',
  '253.529999',
  '247.929993',
  '252.380005',
  '252.380005',
  '8149000'],
 ['2019-07-17',
  '255.669998',
  '258.309998',
  '253.350006',
  '254.860001',
  '254.860001',
  '9764700'],
 ['2019-07-18',
  '255.050003',
  '255.750000',
  '251.889999',
  '253.539993',
  '253.539993',
  '4764500']]

In [21]:
google_rdd = sc.textFile(path_to_data + '/GOOG.csv')
amazon_rdd = sc.textFile(path_to_data + '/AMZN.csv')

In [23]:
header = csv_rdd.first()

In [24]:
#создание дф и добавление названия столбцов из rdd 
tesla_df = csv_rdd.filter(lambda row:row != header).toDF(header)

In [25]:
tesla_df.show(5)

+----------+----------+----------+----------+----------+----------+--------+
|      Date|      Open|      High|       Low|     Close|  AdjClose|  Volume|
+----------+----------+----------+----------+----------+----------+--------+
|2019-07-15|248.000000|254.419998|244.860001|253.500000|253.500000|11000100|
|2019-07-16|249.300003|253.529999|247.929993|252.380005|252.380005| 8149000|
|2019-07-17|255.669998|258.309998|253.350006|254.860001|254.860001| 9764700|
|2019-07-18|255.050003|255.750000|251.889999|253.539993|253.539993| 4764500|
|2019-07-19|255.690002|259.959991|254.619995|258.179993|258.179993| 7048400|
+----------+----------+----------+----------+----------+----------+--------+
only showing top 5 rows



In [27]:
amazon_df  = sqlContext.read.load(path_to_data + '/AMZN.csv', format ='com.databricks.spark.csv', 
                                 header = 'true', 
                                 inferSchema = 'true')

google_df  = sqlContext.read.load(path_to_data + '/GOOG.csv', format ='com.databricks.spark.csv', 
                                 header = 'true', 
                                 inferSchema = 'true')

In [30]:
amazon_df.show(2)

+----------+-----------+-----------+-----------+-----------+-----------+-------+
|      Date|       Open|       High|        Low|      Close|   AdjClose| Volume|
+----------+-----------+-----------+-----------+-----------+-----------+-------+
|2019-07-15|2021.400024|2022.900024|2001.550049| 2020.98999| 2020.98999|2981300|
|2019-07-16|2010.579956|2026.319946|2001.219971|2009.900024|2009.900024|2618200|
+----------+-----------+-----------+-----------+-----------+-----------+-------+
only showing top 2 rows



In [31]:
amazon_df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- AdjClose: double (nullable = true)
 |-- Volume: integer (nullable = true)



In [32]:
amazon_df.count()

253

In [35]:
google_df.show(5)

+----------+-----------+-----------+-----------+-----------+-----------+-------+
|      Date|       Open|       High|        Low|      Close|   AdjClose| Volume|
+----------+-----------+-----------+-----------+-----------+-----------+-------+
|2019-07-15|1146.859985|1150.819946|1139.400024|1150.339966|1150.339966| 903800|
|2019-07-16|     1146.0|1158.579956|     1145.0|1153.579956|1153.579956|1238800|
|2019-07-17|1150.969971|1158.359985| 1145.77002|1146.349976|1146.349976|1170000|
|2019-07-18| 1141.73999| 1147.60498| 1132.72998|1146.329956|1146.329956|1291300|
|2019-07-19|1148.189941|1151.140015|1129.619995|1130.099976|1130.099976|1647200|
+----------+-----------+-----------+-----------+-----------+-----------+-------+
only showing top 5 rows



In [36]:
#более привычное отображение через дф в пандас 
amazon_df.toPandas().head(5)

,Date,Open,High,Low,Close,AdjClose,Volume
0,2019-07-15,2021.400024,2022.900024,2001.550049,2020.989990,2020.989990,2981300
1,2019-07-16,2010.579956,2026.319946,2001.219971,2009.900024,2009.900024,2618200
2,2019-07-17,2007.050049,2012.000000,1992.030029,1992.030029,1992.030029,2558800
3,2019-07-18,1980.010010,1987.500000,1951.550049,1977.900024,1977.900024,3504300
4,2019-07-19,1991.209961,1996.000000,1962.229980,1964.520020,1964.520020,3185600


In [34]:
google_df.show(5)

+----------+-----------+-----------+-----------+-----------+-----------+-------+
|      Date|       Open|       High|        Low|      Close|   AdjClose| Volume|
+----------+-----------+-----------+-----------+-----------+-----------+-------+
|2019-07-15|1146.859985|1150.819946|1139.400024|1150.339966|1150.339966| 903800|
|2019-07-16|     1146.0|1158.579956|     1145.0|1153.579956|1153.579956|1238800|
|2019-07-17|1150.969971|1158.359985| 1145.77002|1146.349976|1146.349976|1170000|
|2019-07-18| 1141.73999| 1147.60498| 1132.72998|1146.329956|1146.329956|1291300|
|2019-07-19|1148.189941|1151.140015|1129.619995|1130.099976|1130.099976|1647200|
+----------+-----------+-----------+-----------+-----------+-----------+-------+
only showing top 5 rows



In [38]:
google_df.select(year('Date').alias('yr'), 'Close').show()

+----+-----------+
|  yr|      Close|
+----+-----------+
|2019|1150.339966|
|2019|1153.579956|
|2019|1146.349976|
|2019|1146.329956|
|2019|1130.099976|
|2019|1138.069946|
|2019|1146.209961|
|2019|1137.810059|
|2019|1132.119995|
|2019|1250.410034|
|2019|1239.410034|
|2019|1225.140015|
|2019|1216.680054|
|2019| 1209.01001|
|2019| 1193.98999|
|2019|1152.319946|
|2019|1169.949951|
|2019| 1173.98999|
|2019|1204.800049|
|2019| 1188.01001|
+----+-----------+
only showing top 20 rows



In [42]:
google_df\
.select(year('Date').alias('yr'), 'Close')\
.filter(google_df['close'] > 1200)\
.show(5)

+----+-----------+
|  yr|      Close|
+----+-----------+
|2019|1250.410034|
|2019|1239.410034|
|2019|1225.140015|
|2019|1216.680054|
|2019| 1209.01001|
+----+-----------+
only showing top 5 rows



In [43]:
google_df\
.select(year('Date').alias('yr'), 'Close')\
.groupby('yr')\
.avg('Close').sort('yr').show()

+----+------------------+
|  yr|        avg(Close)|
+----+------------------+
|2019|1245.3833654621849|
|2020|1362.8286906865671|
+----+------------------+



In [44]:
amazon_df\
.select(year('Date').alias('year'),'Close')\
.groupby('year')\
.min('Close')\
.sort('year')\
.show()

+----+-----------+
|year| min(Close)|
+----+-----------+
|2019| 1705.51001|
|2020|1676.609985|
+----+-----------+



In [ ]:
#план запроса - сам запрос не выполняется в этот момент 
amazon_df\
.select(year('Date').alias('year'),'Close')\
.groupby('year')\
.min('Close')\
.sort('year')\
.explain()

== Physical Plan ==
*(3) Sort [year#236 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(year#236 ASC NULLS FIRST, 200), true, [id=#197]
   +- *(2) HashAggregate(keys=[year#236], functions=[min(Close#38)])
      +- Exchange hashpartitioning(year#236, 200), true, [id=#193]
         +- *(1) HashAggregate(keys=[year#236], functions=[partial_min(Close#38)])
            +- *(1) Project [year(cast(Date#34 as date)) AS year#236, Close#38]
               +- FileScan csv [Date#34,Close#38] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/home/jovyan/work/AMZN.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Date:string,Close:double>




Датафреймы можно сложить во временные таблицы и уже работать с ними через SQLContext. 



In [45]:
amazon_df.registerTempTable("amazon_stocks")
google_df.registerTempTable("google_stocks")
tesla_df.registerTempTable("tesla_stocks")

In [46]:
tesla_df.show()

+----------+----------+----------+----------+----------+----------+--------+
|      Date|      Open|      High|       Low|     Close|  AdjClose|  Volume|
+----------+----------+----------+----------+----------+----------+--------+
|2019-07-15|248.000000|254.419998|244.860001|253.500000|253.500000|11000100|
|2019-07-16|249.300003|253.529999|247.929993|252.380005|252.380005| 8149000|
|2019-07-17|255.669998|258.309998|253.350006|254.860001|254.860001| 9764700|
|2019-07-18|255.050003|255.750000|251.889999|253.539993|253.539993| 4764500|
|2019-07-19|255.690002|259.959991|254.619995|258.179993|258.179993| 7048400|
|2019-07-22|258.750000|262.149994|254.190002|255.679993|255.679993| 6842400|
|2019-07-23|256.709991|260.480011|254.500000|260.170013|260.170013| 5023100|
|2019-07-24|259.170013|266.070007|258.160004|264.880005|264.880005|11072800|
|2019-07-25|233.500000|234.500000|225.550003|228.820007|228.820007|22418300|
|2019-07-26|226.919998|230.259995|222.250000|228.039993|228.039993|10027700|

In [48]:
#sql контекст позваоляет делать запросы на sql из ноутбука
sqlContext.sql('''
SELECT * from GOOGLE_STOCKS LIMIT 5
''').show()

+----------+-----------+-----------+-----------+-----------+-----------+-------+
|      Date|       Open|       High|        Low|      Close|   AdjClose| Volume|
+----------+-----------+-----------+-----------+-----------+-----------+-------+
|2019-07-15|1146.859985|1150.819946|1139.400024|1150.339966|1150.339966| 903800|
|2019-07-16|     1146.0|1158.579956|     1145.0|1153.579956|1153.579956|1238800|
|2019-07-17|1150.969971|1158.359985| 1145.77002|1146.349976|1146.349976|1170000|
|2019-07-18| 1141.73999| 1147.60498| 1132.72998|1146.329956|1146.329956|1291300|
|2019-07-19|1148.189941|1151.140015|1129.619995|1130.099976|1130.099976|1647200|
+----------+-----------+-----------+-----------+-----------+-----------+-------+



In [49]:
sqlContext.sql('''
SELECT year(Date) AS YEAR,
       month(Date) AS MONTH,
       avg(CLOSE)
FROM amazon_stocks
GROUP BY YEAR,
         MONTH
ORDER BY YEAR,
         MONTH
LIMIT 15
''').show()

+----+-----+------------------+
|YEAR|MONTH|        avg(CLOSE)|
+----+-----+------------------+
|2019|    7|1964.6846265384618|
|2019|    8|1793.6027220909093|
|2019|    9|     1799.12099615|
|2019|   10|1752.3317498695653|
|2019|   11|      1774.2939941|
|2019|   12|1785.7728446190476|
|2020|    1|1884.2376128571425|
|2020|    2|2066.1752672631574|
|2020|    3|1872.3104358636365|
|2020|    4|2228.7052408571426|
|2020|    5|2394.1840209499996|
|2020|    6|      2613.5454545|
|2020|    7| 3053.100016222222|
+----+-----+------------------+



In [50]:
sqlContext.sql('''
SELECT t.Date,
       t.Open,
       t.Close,
       t.Open-t.Close AS dif
FROM google_stocks t
WHERE t.Open-t.Close >=2
  OR t.Open-t.Close <= -2
''').show()

+----------+-----------+-----------+-------------------+
|      Date|       Open|      Close|                dif|
+----------+-----------+-----------+-------------------+
|2019-07-15|1146.859985|1150.339966|-3.4799809999999525|
|2019-07-16|     1146.0|1153.579956| -7.579956000000038|
|2019-07-17|1150.969971|1146.349976|  4.619995000000017|
|2019-07-18| 1141.73999|1146.329956| -4.589966000000004|
|2019-07-19|1148.189941|1130.099976|  18.08996500000012|
|2019-07-22|1133.449951|1138.069946| -4.619995000000017|
|2019-07-23|     1144.0|1146.209961| -2.209961000000021|
|2019-07-24|1131.900024|1137.810059|  -5.91003499999988|
|2019-07-25|1137.819946|1132.119995| 5.6999510000000555|
|2019-07-26|1224.040039|1250.410034|-26.369995000000017|
|2019-07-31|     1223.0|1216.680054|  6.319946000000073|
|2019-08-01|1214.030029| 1209.01001| 5.0200190000000475|
|2019-08-02| 1200.73999| 1193.98999|               6.75|
|2019-08-05|1170.040039|1152.319946| 17.720092999999906|
|2019-08-06|1163.310059|1169.94

In [51]:
sqlContext.sql('''
SELECT year(date) AS YEAR,
       max(adjclose),
       min(adjclose)
FROM tesla_stocks
GROUP BY YEAR
''').show()

+----+-------------+-------------+
|YEAR|max(adjclose)|min(adjclose)|
+----+-------------+-------------+
|2019|   430.940002|   211.399994|
|2020|   994.320007|  1000.900024|
+----+-------------+-------------+



In [58]:
joinclose_df = sqlContext.sql('''
WITH tesla AS
  (SELECT year(date) AS YEAR,
          max(adjclose) AS max_tesla,
          min(adjclose) AS min_tesla
   FROM tesla_stocks
   GROUP BY YEAR),
     google AS
  (SELECT year(date) AS YEAR,
          max(adjclose) AS max_google,
          min(adjclose) AS min_google
   FROM google_stocks
   GROUP BY YEAR),
     amazon AS
  (SELECT year(date) AS YEAR,
          max(adjclose) AS max_amazon,
          min(adjclose) AS min_amazon
   FROM amazon_stocks
   GROUP BY YEAR)
SELECT t.YEAR, 
  t.max_tesla, t.min_tesla,
  a.max_amazon, a.min_amazon, 
  g.max_google, g.min_google
FROM tesla t
LEFT JOIN amazon a ON t.year = a.year
LEFT JOIN google g ON t.year = g.year
''')

In [59]:
joinclose_df.show()

+----+----------+-----------+----------+-----------+-----------+-----------+
|YEAR| max_tesla|  min_tesla|max_amazon| min_amazon| max_google| min_google|
+----+----------+-----------+----------+-----------+-----------+-----------+
|2019|430.940002| 211.399994|2020.98999| 1705.51001|1361.170044|1130.099976|
|2020|994.320007|1000.900024|    3200.0|1676.609985| 1541.73999|1056.619995|
+----+----------+-----------+----------+-----------+-----------+-----------+



In [60]:
joinclose_df.write.format('parquet').save('join_stocks.parquet')

In [61]:
final_df = sqlContext.read.parquet('join_stocks.parquet')

In [62]:
final_df.show()

+----+----------+-----------+----------+-----------+-----------+-----------+
|YEAR| max_tesla|  min_tesla|max_amazon| min_amazon| max_google| min_google|
+----+----------+-----------+----------+-----------+-----------+-----------+
|2020|994.320007|1000.900024|    3200.0|1676.609985| 1541.73999|1056.619995|
|2019|430.940002| 211.399994|2020.98999| 1705.51001|1361.170044|1130.099976|
+----+----------+-----------+----------+-----------+-----------+-----------+

